In [29]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Input
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix, f1_score, classification_report, accuracy_score
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig, TFDistilBertModel, DistilBertTokenizer, \
    DistilBertConfig

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, StratifiedKFold
from sklearn.metrics import confusion_matrix

In [30]:
df_train = pd.read_csv(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\treino\r2_bo_train_statements.csv', sep=';')
df_train = df_train.drop(df_train[df_train.Polarity =='other'].index)
df_train.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,Precisamos urgente continuar com a retrospecti...,r2_bo,0,no,r2_bo_21,_1088973003227103239,471,"[('precisamos', 'V'), ('urgente', 'ADJ'), ('co...",[],"{'futuro': 1, 'bolsonaro': 1, 'retrospectiva':..."
1,Oh criança.... Vc é da democracia? ENTÃO... te...,r2_bo,1,no,r2_bo_728,_1057063517608648704,1642,"[('oh', 'IN'), ('criança', 'N'), ('...', 'PU')...",['jairbolsonaro'],"{'democracia': 3, 'escolha': 1, 'povo': 1, 'de..."
2,federal não é balburdia não governo Bolsonaro ...,r2_bo,0,no,r2_bo_667,_1168379660805255170,4752,"[('federal', 'ADJ'), ('não', 'ADV'), ('é', 'V'...",[],{}
3,bolsonaro Pro teu pai desejo o dobro de tudo q...,r2_bo,0,no,r2_bo_73,_1280338007476895749,1055,"[('@', 'NUM'), ('carlosbolsonaro', 'N'), ('pro...",['carlosbolsonaro'],"{'carlosbolsonaro': 1, 'dobro': 1, 'presidente..."
4,Nunca achei que fosse gostar de ver o bolsonaro,r2_bo,1,no,r2_bo_614,_1323469943510360064,929,"[('@', 'NUM'), ('ittsroberta', 'N'), ('nunca',...",['ittsroberta'],"{'bolsonaro': 1, 'ittsroberta': 1}"
...,...,...,...,...,...,...,...,...,...,...
7055,Bolsonaro libera compra de munição para que o ...,r2_bo,0,no,r2_bo_75,_1253347442319376387,4174,"[('bolsonaro', 'N'), ('libera', 'V'), ('compra...",[],"{'coronavírus': 1, 'bolsonaro': 1, 'munição': ..."
7056,"Bolsonaro cometeu falsidade ideológica”, diz c...",r2_bo,0,yes,r2_bo_75,_1253998970453532672,4271,"[('“', 'N'), ('bolsonaro', 'N'), ('cometeu', '...",[],"{'bolsonaro': 1, 'criminalista': 1, 'falsidade..."
7057,NÃO TEM ExÉRCITO QUE DÊ CONTA DO CAOS GENERALI...,r2_bo,1,no,r2_bo_702,_1242855135107301379,3784,"[('não', 'ADV'), ('tem', 'V'), ('exército', 'N...",[],"{'caos': 2, 'país': 1, 'bolsonaro': 1, 'conta'..."
7058,bolsonaro antes disseminava vários discursos d...,r2_bo,0,no,r2_bo_309,_904371719787474945,1861,"[('bolsonaro', 'ADV'), ('antes', 'ADV'), ('dis...",[],"{'eleição': 1, 'discursos': 1, 'ódio': 1}"


In [31]:
df_test = pd.read_csv(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\Teste\r2_bo_test_statements.csv', sep=';')
df_test = df_test.drop(df_test[df_test.Polarity =='other'].index)
df_test.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,Na tentativa de esculachar Bolsonaro e acusa l...,r2_bo,1,no,r2_bo_716,_1092400920934182915,964,"[('sim', 'ADV'), ('foi', 'V'), ('para', 'PREP'...",[],"{'culatra': 1, 'marginais': 1, 'bolsonaro': 1,..."
1,o pior está ppr vir que bolsonaro vai falar ja ja,r2_bo,0,no,r2_bo_464,_1253758870914301953,5018,"[('o', 'ART'), ('pior', 'N'), ('está', 'V'), (...",[],{'pior': 1}
2,Nossa que engraçado vc reclamando da violência...,r2_bo,0,no,r2_bo_61,_1221080812759789568,3991,"[('nossa', 'PROADJ'), ('que', 'KS'), ('engraça...",[],"{'violência': 1, 'orgulho': 1, 'bolsonaro': 1,..."
3,Qualquer um pode!!O Presidente da Câmara é rei...,r2_bo,1,no,r2_bo_702,_1131245002301038597,1450,"[('qualquer', 'PROADJ'), ('um', 'PROSUB'), ('p...",[],"{'cabeças': 1, 'críticas': 1, 'bolsonaro': 1, ..."
4,OCDE sinaliza que não aceitará o Brasil de Bol...,r2_bo,0,no,r2_bo_75,_1258316008022061056,4610,"[('ocde', 'N'), ('sinaliza', 'V'), ('que', 'KS...",['a_postagem'],"{'a_postagem': 1, 'ocde': 1, 'brasil': 1}"
...,...,...,...,...,...,...,...,...,...,...
2349,"Sr General e Vice Presidente Mourão, eu votei ...",r2_bo,1,no,r2_bo_510,_1239986133523709955,2819,"[('sr', 'N'), ('general', 'N'), ('e', 'KC'), (...",[],"{'presidente': 3, 'sr': 3, 'general': 2, 'pess..."
2350,O Brasil se afastando do 1º mundo. Deputados d...,r2_bo,0,yes,r2_bo_753,_1052201670485004288,838,"[('o', 'ART'), ('brasil', 'N'), ('se', 'PROPES...",['tvi24'],"{'manifesto': 1, 'brasil': 1, 'deputados': 1, ..."
2351,Que vergonha já ter usado Karin Strobel como r...,r2_bo,0,no,r2_bo_677,_1243197965122093056,3456,"[('que', 'PRO-KS'), ('vergonha', 'N'), ('já', ...",[],"{'secretaria': 1, 'faculdade': 1, 'pior': 1, '..."
2352,e essa notícia que roubaram a moto e a arma do...,r2_bo,0,no,r2_bo_183,_980119243785232385,1792,"[('e', 'KC'), ('essa', 'PROADJ'), ('notícia', ...",[],"{'bolsonaro': 1, 'arma': 1, 'moto': 1, 'mano':..."


In [32]:
#carregando unlabeled data
df_u = pd.read_csv(r'C:/Users/camil/OneDrive/Área de Trabalho/mestrado/Unlabeled/bolsonaro.csv')
#df_u = df.sample(frac=1).reset_index(drop=True)
df_u

,Unnamed: 0,Text
0,0,isso me lembra eu anto e bruna discutindo c o ...
1,1,bolsonaro fdp acabando c a maconha esse país t...
2,2,Bolsonaro 2018? — No dia das eleições eu ainda...
3,3,Eu amo o Bolsonaro
4,4,acabou os seiscentao do bolsonaro entao amanha...
...,...,...
31990,31990,Eu conto 0 items dessa lista no governo Bolson...
31991,31991,Não existe comparação nenhuma entre os últimos...
31992,31992,Sendo sincero: seria injusto até comparar a ca...
31993,31993,Eu vou ser sinceros com vcs: eu não acho que n...


In [33]:
tokenizer = AutoTokenizer.from_pretrained("pablocosta/bertabaporu-base-uncased")
BRmodel = AutoModelForMaskedLM.from_pretrained("pablocosta/bertabaporu-base-uncased")
BRmodel.config.output_attentions


Some weights of the model checkpoint at pablocosta/bertabaporu-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


False

In [34]:
from simpletransformers.classification import ClassificationModel
import torch

args={'reprocess_input_data': True,'overwrite_output_dir': True, 'num_train_epochs': 1}

cuda_available = torch.cuda.is_available()

model = ClassificationModel('bert', 'pablocosta/bertabaporu-base-uncased',use_cuda=cuda_available, args=args)

Some weights of the model checkpoint at pablocosta/bertabaporu-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification

In [35]:
df_train = df_train[['Text', 'Polarity']]
df_test = df_test[['Text', 'Polarity']]

In [36]:
model.train_model(df_train)

C:\Users\camil\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:601: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/7060 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/883 [00:00<?, ?it/s]

(883, 0.4556122627594463)

In [37]:
result, model_outputs, wrong_predictions = model.eval_model(df_test)

C:\Users\camil\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:1442: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2354 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/295 [00:00<?, ?it/s]

In [38]:
result

{'mcc': 0.661237619467726,
 'tp': 766,
 'tn': 1203,
 'fp': 189,
 'fn': 196,
 'auroc': 0.9083114530312806,
 'auprc': 0.8806837670954478,
 'eval_loss': 0.4066872615998579}

In [39]:
pred_class = np.argmax(model_outputs, axis=-1)
np.savetxt(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\pred_bo_Btweet_rotulado.csv', pred_class, delimiter=";")

In [40]:
f1_score(df_test['Polarity'], pred_class)

0.7991653625456442

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(df_test['Polarity'], pred_class)

0.8364485981308412

In [42]:
predictions, raw_outputs = model.predict(df_u['Text'].values.tolist())

  0%|          | 0/31995 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [43]:
np.savetxt(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\bo_unlabel_pred_0.csv', predictions, delimiter=";")
df_u.to_csv(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\bo_unlabel_text_0.csv')

In [44]:
rows_unlabel = len(df_u.index)
rows_unlabel

31995

In [45]:
threshold = 0.01
t_size = int(threshold*rows_unlabel) #quanto vai adicionar a cada iteração

In [46]:
reliable_pred_favor, reliable_pred_against  = [], []
for x_id, (prob, pred) in enumerate(zip(raw_outputs, predictions)):
    #se 1, inclui no favor; senão, no against
    #incluir id, probabilidade e valor predito (1 ou 0)
    (reliable_pred_favor if (predictions[x_id]) else reliable_pred_against).append((x_id, raw_outputs[x_id,predictions[x_id]], predictions[x_id]))
    

In [47]:
#coloca em ordem decrescente de confiabilidade a partir da probabilidade (coluna 2)
reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)

In [48]:
list_index=[]
position = 0
#lista todos os índices para remover do dataframe
for i in range(int(t_size/2)):
        #pega um a favor do topo
        index = reliable_pred_favor[position][0] #descobre a posição no X_label 
        list_index.append(index) #inclui na lista de indices
        df_train.loc[len(df_train.index)] = [df_u.iloc[index], 1]#adiciona linha no treinamento
        #pega um contra do topo
        index = reliable_pred_against[position][0] #descobre a posição no X_label 
        list_index.append(index) #inclui na lista de indices
        df_train.loc[len(df_train.index)] = [df_u.iloc[index], 0]#adiciona linha no treinamento
        #anda uma posição 
        position+=1
       

C:\Users\camil\AppData\Local\Temp\ipykernel_15460\4278097114.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[len(df_train.index)] = [df_u.iloc[index], 1]#adiciona linha no treinamento
C:\Users\camil\AppData\Local\Temp\ipykernel_15460\4278097114.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[len(df_train.index)] = [df_u.iloc[index], 0]#adiciona linha no treinamento
C:\Users\camil\AppData\Local\Temp\ipykernel_15460\4278097114.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [49]:
#remove índices usados do df_u
df_u = df_u.drop(list_index)
df_u.reset_index(drop=True, inplace=True)
df_u

,Unnamed: 0,Text
0,0,isso me lembra eu anto e bruna discutindo c o ...
1,1,bolsonaro fdp acabando c a maconha esse país t...
2,2,Bolsonaro 2018? — No dia das eleições eu ainda...
3,3,Eu amo o Bolsonaro
4,4,acabou os seiscentao do bolsonaro entao amanha...
...,...,...
31672,31990,Eu conto 0 items dessa lista no governo Bolson...
31673,31991,Não existe comparação nenhuma entre os últimos...
31674,31992,Sendo sincero: seria injusto até comparar a ca...
31675,31993,Eu vou ser sinceros com vcs: eu não acho que n...


In [50]:
#atualiza o modelo
args={'reprocess_input_data': True,'overwrite_output_dir': True, 'num_train_epochs': 1}
cuda_available = torch.cuda.is_available()
model = ClassificationModel('bert', 'pablocosta/bertabaporu-base-uncased',use_cuda=cuda_available, args=args)

Some weights of the model checkpoint at pablocosta/bertabaporu-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification

In [51]:
#atualiza o modelo
model.train_model(df_train)

C:\Users\camil\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:601: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/7378 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/923 [00:00<?, ?it/s]

(923, 0.4426538680305751)

In [52]:
result, model_outputs, wrong_predictions = model.eval_model(df_test)

C:\Users\camil\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:1442: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2354 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/295 [00:00<?, ?it/s]

In [53]:
pred_class = np.argmax(model_outputs, axis=-1)
np.savetxt(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\pred_bo_Btweet_01.csv', pred_class, delimiter=";")

In [54]:
f1_score(df_test['Polarity'], pred_class)

0.8035620743844945

In [55]:
accuracy_score(df_test['Polarity'], pred_class)

0.840696686491079

In [56]:
#adiciona até 5x 0.01
iteractions = 0
while iteractions < 5:
    iteractions+=1
    predictions, raw_outputs = model.predict(df_u['Text'].values.tolist())
    path_predictions = r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\bo_unlabel_' +str(iteractions)+'.csv'
    path_unlabel = r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\bo_unlabel_text_' +str(iteractions)+'.csv'
    np.savetxt(path_predictions, predictions, delimiter=";")
    df_u.to_csv(path_unlabel)
    rows_unlabel = len(df_u.index)
    reliable_pred_favor, reliable_pred_against  = [], []
    for x_id, (prob, pred) in enumerate(zip(raw_outputs, predictions)):
        #se 1, inclui no favor; senão, no against
        #incluir id, probabilidade e valor predito (1 ou 0)
        (reliable_pred_favor if (predictions[x_id]) else reliable_pred_against).append((x_id, raw_outputs[x_id,predictions[x_id]], predictions[x_id]))
    
    #coloca em ordem decrescente de confiabilidade a partir da probabilidade (coluna 2)
    reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
    reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)
    list_index=[]
    position = 0
    #lista todos os índices para remover do dataframe
    for i in range(int(t_size/2)):
        #pega um a favor do topo
        index = reliable_pred_favor[position][0] #descobre a posição no X_label 
        list_index.append(index) #inclui na lista de indices
        df_train.loc[len(df_train.index)] = [df_u.iloc[index], 1]#adiciona linha no treinamento
        #pega um contra do topo
        index = reliable_pred_against[position][0] #descobre a posição no X_label 
        list_index.append(index) #inclui na lista de indices
        df_train.loc[len(df_train.index)] = [df_u.iloc[index], 0]#adiciona linha no treinamento
        #anda uma posição 
        position+=1
    df_u = df_u.drop(list_index)
    df_u.reset_index(drop=True, inplace=True)
    #atualiza o modelo
    args={'reprocess_input_data': True,'overwrite_output_dir': True, 'num_train_epochs': 1}
    cuda_available = torch.cuda.is_available()
    model = ClassificationModel('bert', 'pablocosta/bertabaporu-base-uncased',use_cuda=cuda_available, args=args)
    model.train_model(df_train)    
    result, model_outputs, wrong_predictions = model.eval_model(df_test)
    pred_class = np.argmax(model_outputs, axis=-1)
    f1 = f1_score(df_test['Polarity'], pred_class)
    acc = accuracy_score(df_test['Polarity'], pred_class)
    print("iteraction:",iteractions, "accuracy:", acc, "f1_score:", f1)
    path = r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\pred_bo_Btweet_'+ str(iteractions +1)+'.csv'
    np.savetxt(path, pred_class, delimiter=";")


  0%|          | 0/31677 [00:00<?, ?it/s]

  0%|          | 0/3960 [00:00<?, ?it/s]

C:\Users\camil\AppData\Local\Temp\ipykernel_15460\2979591486.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[len(df_train.index)] = [df_u.iloc[index], 1]#adiciona linha no treinamento
C:\Users\camil\AppData\Local\Temp\ipykernel_15460\2979591486.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[len(df_train.index)] = [df_u.iloc[index], 0]#adiciona linha no treinamento
C:\Users\camil\AppData\Local\Temp\ipykernel_15460\2979591486.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

  0%|          | 0/7696 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/1121 [00:00<?, ?it/s]

C:\Users\camil\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:1442: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2354 [00:01<?, ?it/s]

Running Evaluation:   0%|          | 0/295 [00:00<?, ?it/s]

iteraction: 5 accuracy: 0.8377230246389125 f1_score: 0.7937365010799136
